In [29]:
import pandas as pd
import numpy as np
import pylab as pl
import scipy.io as sio
import seaborn as sns
import statsmodels.api as sm
import funcs as func
import sys
import pdb
import glob
import matplotlib.pyplot as plt
import os

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

cwd = os.getcwd()

print(cwd)
data_dir = cwd+"/Data/Y_maze/data_with_animal_ids/"
data_target_dir = cwd+"/Data/processed_data/Y_maze/data_with_animal_ids/"
figure_dir = cwd+"/Figures/"


/home/jyotika/2-arm-bandit-task


In [30]:
#experiment_type = "NoConfLowVolDTs"

plt.rcParams["figure.facecolor"] = "w"

In [31]:
col_names = ["time (ms)","event_marker","current_trial_num_in_block","block","low_rew_%","high_rew_%","low_trial_range(block)","high_trial_range(block)","block#","rewarded_trial_count","total_trial_count","sig","x_pos","y_pos","trip","chnc","current_hall","previous_hall","lick_rate_hall1","lick_rate_hall2","lick_rate_hall3","wtr","mouse_distance","TTL"]

non_action_em = [0,11,12,9,10]
left_action_em = [1,2,3,4]
right_action_em = [5,6,7,8]
rewarded_em = [1,3,5,7]
unrewarded_em = [2,4,6,8]

In [8]:
def read_raw_data(mat,ids):
    # block, 0 = left, 1 = right
    #columns = ["time (ms)", "event_markers", "current_trial_num_in_block","block","reward_probability","conflict","volatility","block_num","rewarded","action_chosen","condition"]
    r,c = np.shape(mat[0])
    raw_df = pd.DataFrame(columns = list(np.arange(0,c,1))+["session"])
    ids1 = np.hstack(ids)[0]
    print(ids1)
    for i in np.arange(len(mat)):
        temp = pd.DataFrame(mat[i])
        temp["session"] = i
        temp["animal_id"] = ids1[i]
        raw_df = raw_df.append(temp)

    return raw_df



def process_raw(raw,experiment_type):
    processed_df = pd.DataFrame(columns=["conflict","condition","volatility","event_marker","block","reward_probability","trial_length_range","session","block_num","trial_num","chosen_action","rewarded","animal_id","chnc"])
    conf = experiment_type.split('Conf')[0]
    cond = experiment_type.split('Vol')[1]
    #cond = "Control" # change when stim data is here !!!!!!
    vol = experiment_type.split('Vol')[0].split('Conf')[1]
    print(conf,cond,vol)
    for grp in raw.groupby(["session","block#","current_trial_num_in_block","animal_id"]):
#         print(grp[0])
        temp = dict()
        temp["session"] = grp[0][0]
        temp["animal_id"] = int(grp[0][3])
        if temp["animal_id"]%2 == 0 and cond == "Stim": # even animal id
            cond1 = "dSPN-Stim"
        elif temp["animal_id"]%2 != 0 and cond == "Stim": # odd animal id
            cond1 = "iSPN-Stim"
        elif temp["animal_id"]%2 == 0 and cond == "Control": # even animal id
            cond1 = "dSPN-Control"
        elif temp["animal_id"]%2 != 0 and cond == "Control": # odd animal id
            cond1 = "iSPN-Control"
#         else:
#             cond1 = cond
        temp["conflict"] = conf
        temp["volatility"] = vol
        temp["condition"] = cond1
        temp["block_num"] = grp[0][1]
        temp["trial_num"] = grp[0][2]
#         time_before_iti = grp[1].loc[grp[1]["event_marker"]!=12]
#         temp["RT(ms)"] = np.max(time_before_iti["time (ms)"]) - np.min(time_before_iti["time (ms)"])
        action_em = list(set(np.unique(grp[1]["event_marker"]))-set(non_action_em))
        #print(action_em)
        if len(action_em) == 1:
            ind_action_em = np.where(grp[1]["event_marker"]==action_em[0])[0]
            uniq_chnc = np.unique(grp[1].iloc[ind_action_em]["chnc"])
            if len(uniq_chnc) > 1:
                nz = np.where(uniq_chnc!=0)
                uniq_chnc = uniq_chnc[nz]
            #print(ind_action_em)
            else:
                uniq_chnc 
            
            temp["chnc"] = uniq_chnc
            
            #print(temp["chnc"])
        #print(action_em)       
        if len(action_em) == 1:
            temp["event_marker"] = action_em[0]
            if action_em[0] in left_action_em:
                temp["chosen_action"] = "left"
            elif action_em[0] in right_action_em:
                temp["chosen_action"] = "right"
            if action_em[0] in rewarded_em:
                temp["rewarded"] = "rewarded"
            elif action_em[0] in unrewarded_em:
                temp["rewarded"] = "unrewarded"
        else:
            #print("no action ems",np.unique(grp[1]["event_marker"]))
            #print(grp[0])
            temp["event_marker"] = np.nan
            temp["reward_probability"] = np.nan
            temp["chosen_action"] = np.nan
            temp["rewarded"] = np.nan

        temp["block"] = "left" if np.unique(grp[1]["block"])[0] == 0 else "right"
        temp["trial_length_range"] = str(np.unique(grp[1]["low_trial_range(block)"])[0])+"-"+str(np.unique(grp[1]["high_trial_range(block)"])[0])

        temp["reward_probability"] = str(np.unique(grp[1]["low_rew_%"])[0])+"-"+str(np.unique(grp[1]["high_rew_%"])[0])
        temp["optimal_action"] = 1 if temp["block"] == temp["chosen_action"] else 0
        
        processed_df = processed_df.append(temp,ignore_index=True)

    return processed_df



In [9]:
# exp_files = glob.glob(data_dir+"*.mat")
# print(exp_files)
# for ef in exp_files:
#     print(ef)
#     exp_type = ef.split('/')[-1].split('.')[0]
#     exp_mat = sio.loadmat(ef)


In [10]:
# ef = '/home/jyotika/2-arm-bandit-task/Data/Y_maze/data_with_animal_ids/NoConfHighVolStim_withDTs.mat'
# exp_type = ef.split('/')[-1].split('.')[0].split('_')[0]
# print(exp_type)
# exp_mat = sio.loadmat(ef)
# raw_df = read_raw_data(exp_mat['overlap'][0],exp_mat['overlap'][2])


In [11]:
#     raw_df = raw_df.rename(columns={i:cn for i,cn in zip(np.arange(len(col_names)),col_names)})
#     processed_df = process_raw(raw_df,exp_type)


In [12]:

all_conflicts = pd.DataFrame()
just_rts = pd.DataFrame()
exp_files = glob.glob(data_dir+"*.mat")
print(exp_files)
for ef in exp_files:
    print(ef)
    exp_type = ef.split('/')[-1].split('.')[0].split('_')[0]
    print(exp_type)
    exp_mat = sio.loadmat(ef)
    raw_df = read_raw_data(exp_mat['overlap'][0],exp_mat['overlap'][2])
    # Replace the column numbers with the labels that Julia gave
    raw_df = raw_df.rename(columns={i:cn for i,cn in zip(np.arange(len(col_names)),col_names)})
    processed_df = process_raw(raw_df,exp_type)
    processed_df = processed_df.dropna()
    processed_df = processed_df.reset_index()
    processed_df["index"] = processed_df.index

    
    rt_df = read_raw_data(exp_mat['overlap'][1],exp_mat['overlap'][2])
    rt_df = rt_df.rename(columns ={0:"RT(ms)"})
    rt_df = rt_df.reset_index()
    rt_df["index"] = rt_df.index
    
    final_df = pd.DataFrame()
    for grp1,grp2 in zip(processed_df.groupby(["animal_id"]), rt_df.groupby(["animal_id"])):
        grp1[1]["revised_session_id"] = list(grp1[1]["session"] - np.min(grp1[1]["session"]))
        grp2[1]["revised_session_id"] = list(grp2[1]["session"] - np.min(grp2[1]["session"]))
        
        temp = grp1[1].merge(grp2[1],on=["session","animal_id","index"])
        final_df = final_df.append(temp)
    
#     final_df = processed_df.merge(rt_df,on=["session","animal_id","index"])
    final_df.to_csv(data_target_dir+exp_type+"_processed.csv")
    
#     rt_df["exp_type"] = exp_type
#     just_rts = just_rts.append(rt_df)
    all_conflicts = all_conflicts.append(final_df)
    
all_conflicts = all_conflicts.reset_index()    

['/home/jyotika/2-arm-bandit-task/Data/Y_maze/data_with_animal_ids/LowConfHighVolControl_withDTs.mat', '/home/jyotika/2-arm-bandit-task/Data/Y_maze/data_with_animal_ids/NoConfHighVolStim_withDTs.mat', '/home/jyotika/2-arm-bandit-task/Data/Y_maze/data_with_animal_ids/LowConfHighVolStim_withDTs.mat', '/home/jyotika/2-arm-bandit-task/Data/Y_maze/data_with_animal_ids/LowConfLowVolControl_withDTs.mat', '/home/jyotika/2-arm-bandit-task/Data/Y_maze/data_with_animal_ids/NoConfHighVolControl_withDTs.mat', '/home/jyotika/2-arm-bandit-task/Data/Y_maze/data_with_animal_ids/HighConfLowVolStim_withDTs.mat', '/home/jyotika/2-arm-bandit-task/Data/Y_maze/data_with_animal_ids/LowConfLowVolStim_withDTs.mat', '/home/jyotika/2-arm-bandit-task/Data/Y_maze/data_with_animal_ids/HighConfHighVolStim_withDTs.mat', '/home/jyotika/2-arm-bandit-task/Data/Y_maze/data_with_animal_ids/NoConfLowVolControl_withDTs.mat', '/home/jyotika/2-arm-bandit-task/Data/Y_maze/data_with_animal_ids/NoConfLowVolStim_withDTs.mat', '/ho

In [13]:
# exp_mat = sio.loadmat("/home/jyotika/2-arm-bandit-task/Data/Y_maze/data_with_animal_ids/NoConfHighVolControl_withDTs.mat")
# mat = exp_mat['overlap'][0]
# ids = exp_mat['overlap'][2]
# r,c = np.shape(mat[0])
# raw_df = pd.DataFrame(columns = list(np.arange(0,c,1))+["session"])
# ids1 = np.hstack(ids)[0]
# print(ids1)


In [14]:
# raw_df = read_raw_data(exp_mat['overlap'][0],exp_mat['overlap'][2])

In [15]:
# raw_df = raw_df.rename(columns={i:cn for i,cn in zip(np.arange(len(col_names)),col_names)})

In [16]:
# test = grp[1][(grp[1]["session"]==44)&(grp[1]["block#"]==19)&(grp[1]["current_trial_num_in_block"]==17)&(grp[1]["animal_id"]==8.0)]

In [17]:
# test

In [18]:
# grp[1].loc[grp[1]["event_marker"].isin(left_action_em+right_action_em)][["event_marker","chnc"]]

In [19]:
# np.unique(test["event_marker"])

In [20]:
# sio.savemat("/home/jyotika/2-arm-bandit-task/Data/Y_maze/data_with_animal_ids/NoConfHighVolControl_withDTs.mat",exp_mat)

In [28]:
all_conflicts["rewarded_code"] = [ 1 if x == "rewarded" else 0 for x in all_conflicts["rewarded"]]
all_conflicts["animal_id"] = [ 10.0 if x == 3.0 or x == 3 else x for x in all_conflicts["animal_id"] ]
all_conflicts["chnc"] =  all_conflicts["chnc"].astype(int)
all_conflicts["conflict + volatility"] = all_conflicts["conflict"]+" - "+all_conflicts["volatility"]
all_conflicts.to_csv(data_target_dir+"all_experiments_df_with_DTs.csv")


In [32]:
all_conflicts = pd.read_csv(data_target_dir+"all_experiments_df_with_DTs.csv")

# final_df["rewarded_code"] = [ 1 if x == "rewarded" else 0 for x in final_df["rewarded"]]

In [33]:
sns.catplot(x="conflict",y="ideal_B",hue="condition",data=all_conflicts,kind='point')

ValueError: Could not interpret input 'ideal_B'

In [38]:
np.unique(all_conflicts["condition"])

array(['dSPN-Control', 'dSPN-Stim', 'iSPN-Control', 'iSPN-Stim'],
      dtype=object)

In [39]:
# Replace #3 to #10 - note by Julia
final_df["animal_id"] = [ 10.0 if x == 3.0 else x for x in final_df["animal_id"] ]

In [40]:
np.unique(final_df["animal_id"])

array([ 1,  2,  4,  5,  6,  7,  8,  9, 10])

In [50]:
type(all_conflicts["chnc"])

pandas.core.series.Series

In [42]:
# final_df = processed_df.merge(rt_df,on=["session","animal_id","index"])

In [43]:
final_df = final_df.reset_index()


In [44]:
all_conflicts[["chnc","chosen_action","rewarded"]]

,chnc,chosen_action,rewarded
0,[32],left,rewarded
1,[54],left,rewarded
2,[21],left,rewarded
3,[8],left,rewarded
4,[10],right,unrewarded
...,...,...,...
299830,[61],right,unrewarded
299831,[39],left,rewarded
299832,[53],left,rewarded
299833,[54],left,rewarded


In [ ]:
# just_rts["conflict"] = [x.split('Conf')[0]  for x in just_rts["exp_type"]]
# just_rts["volatility"] = [x.split('Vol')[0].split('Conf')[1]  for x in just_rts["exp_type"]]


In [ ]:
# just_rts["animal_id"] = just_rts["animal_id"].astype(str)

In [ ]:
# for grp in all_conflicts.groupby(["animal_id"]):
#     g2 = sns.catplot(x="conflict",y="optimal_action",hue="condition",data=grp[1],kind='point',order=["No","Low","High"],hue_order=["Control","D1-Stim","D2-Stim"],col='volatility',col_order=["Low","High"])
#     g2.figure.suptitle("animal_id: "+str(grp[0]))
#     g2.figure.subplots_adjust(top=0.93)
#     g2.savefig(figure_dir+"conflict_vs_performance_vs_condition_animal_id_"+str(grp[0])+".png")

In [ ]:
# plt.rc_context({'lines.linewidth': 3.5})
for grp in all_conflicts.groupby(["animal_id"]):
    fig,ax = pl.subplots(2,2,figsize=(24,20))
    rts_range=[]
    perf_range =[]
    for i,vol in enumerate(["Low","High"]):
        gp1 = grp[1].loc[grp[1]["volatility"]==vol]
        ax[i][0].set_title("volatility = "+vol,fontsize=20)
        sns.pointplot(x="conflict",y="optimal_action",hue="condition",data=gp1,ax=ax[i][0],order=["No","Low","High"],hue_order=["dSPN-Control","iSPN-Control","dSPN-Stim","iSPN-Stim"],scale=1.5)
        perf_range.append(ax[i][0].get_ylim())
        ax[i][1].set_title("volatility = "+vol,fontsize=20)
        sns.pointplot(x="conflict",y="RT(ms)",hue="condition",data=gp1,ax=ax[i][1],order=["No","Low","High"],hue_order=["dSPN-Control","iSPN-Control","dSPN-Stim","iSPN-Stim"],scale=1.5)        
        rts_range.append(ax[i][1].get_ylim())

    rts_min,rts_max = np.min(np.hstack(rts_range)),np.max(np.hstack(rts_range))
    p_min,p_max = np.min(np.hstack(perf_range)),np.max(np.hstack(perf_range))
    for i,vol in enumerate(["Low","High"]):
        ax[i][0].set_ylim(p_min,p_max)
        ax[i][1].set_ylim(rts_min,rts_max)
    for x in np.hstack(ax):
        x.set_ylabel(x.get_ylabel(),fontsize=20)
        x.set_ylabel(x.get_ylabel(),fontsize=20)
        for xt in x.get_xticklabels():
            xt.set_fontsize(15)
        for yt in x.get_yticklabels():
            yt.set_fontsize(15)
            
        
    fig.suptitle("animal_id: "+str(grp[0]))
    
    fig.subplots_adjust(top=0.95,left=0.06,right=0.96,bottom=0.06)
    fig.savefig(figure_dir+"conflict_volatility_condition_RT_performance_animal_"+str(grp[0])+".png")

In [ ]:
palette=['firebrick','red','royalblue','cyan']
g1 = sns.catplot(x="conflict",y="optimal_action",hue="condition",col='volatility',order=["No","Low","High"],hue_order=["iSPN-Control","iSPN-Stim","dSPN-Control","dSPN-Stim"],col_order=["Low","High"],data=all_conflicts,kind='bar',palette=palette,height=10,aspect=1)
g1.figure.savefig(figure_dir+"summary_figure_performance.png")

In [ ]:
palette=['firebrick','red','royalblue','cyan']
g1 = sns.catplot(x="conflict",y="RT(ms)",hue="condition",col='volatility',order=["No","Low","High"],hue_order=["iSPN-Control","iSPN-Stim","dSPN-Control","dSPN-Stim"],col_order=["Low","High"],data=all_conflicts,kind='bar',palette=palette,height=10,aspect=1)
g1.figure.savefig(figure_dir+"summary_figure_rt.png")

In [ ]:
# for grp in all_conflicts.groupby(["animal_id"]):
#     g1 = sns.catplot(x="conflict",y="RT(ms)",hue="condition",data=grp[1],kind='point',order=["No","Low","High"],hue_order=["Control","D1-Stim","D2-Stim"],col='volatility',col_order=["Low","High"])
#     g1.figure.suptitle("animal_id: "+str(grp[0]))
#     g1.figure.subplots_adjust(top=0.93)
#     g1.savefig(figure_dir+"conflict_vs_rt_vs_condition_animal_id_"+str(grp[0])+".png")

In [ ]:
g1.figure.axes[1]

In [ ]:
all_conflicts = pd.read_csv(data_target_dir+"all_experiments_df_with_DTs.csv")

In [ ]:
all_conflicts

In [ ]:
# fig,ax = pl.subplots(1,1,figsize=(10,8))
# # sns.histplot(x="RT(ms)",data=all_conflicts,hue='conflict + volatility',log_scale=True,element='step',ax=ax,kde=True,alpha=0.05,stat='density',line_kws={'linewidth':4.5})
# sns.histplot(x="RT(ms)",data=all_conflicts,element='step',ax=ax,alpha=0.05,stat='probability',line_kws={'linewidth':4.5},hue='animal_id',palette='tab10')
# ax.set_xlim(0,8000)
# fig.savefig(figure_dir+"Exanple_RT_distribution.png")

In [ ]:
all_conflicts.loc[all_conflicts["RT(ms)"]==np.max(all_conflicts["RT(ms)"])]

In [ ]:
col_order = [x+" - "+y  for y in ["Low","High"]  for x in ["No","Low","High"]]
# g2 = sns.displot(x="RT(ms)",hue="optimal_action",col='conflict + volatility',data=all_conflicts,col_wrap=3,alpha=0.1,line_kws={'linewidth':4.5},col_order=col_order,multiple='dodge',element='step',bins=10)
# for ax in g2.axes:
#     ax.set_xlim(0,8000)
# g2.savefig(figure_dir+"Correct_vs_incorrect_action_RTs.png")

In [ ]:
g3 = sns.displot(x="RT(ms)",hue="rewarded",col='conflict + volatility',data=all_conflicts.loc[all_conflicts["condition"]=="Control"],col_wrap=3,element='step',alpha=0.05,stat='density',line_kws={'linewidth':4.5})
for ax in g3.axes:
    ax.set_xlim(0,5000)

g3.savefig(figure_dir+"Rewarded_vs_unrewarded_RTs_control.png")

In [ ]:
g3 = sns.displot(x="RT(ms)",hue="rewarded",col='conflict + volatility',data=all_conflicts.loc[all_conflicts["condition"]=="D1-Stim"],col_wrap=3,element='step',alpha=0.05,stat='density',line_kws={'linewidth':4.5})
for ax in g3.axes:
    ax.set_xlim(0,5000)

g3.savefig(figure_dir+"Rewarded_vs_unrewarded_RTs_D1_stim.png")

In [ ]:
g3 = sns.displot(x="RT(ms)",hue="rewarded",col='conflict + volatility',data=all_conflicts.loc[all_conflicts["condition"]=="D2-Stim"],col_wrap=3,element='step',alpha=0.05,stat='density',line_kws={'linewidth':4.5})
for ax in g3.axes:
    ax.set_xlim(0,5000)

g3.savefig(figure_dir+"Rewarded_vs_unrewarded_RTs_D2_stim.png")

In [ ]:

from statannot import add_stat_annotation

ctrl_data = all_conflicts.loc[all_conflicts["condition"]=="Control"]
ctrl_data = ctrl_data.dropna()
ctrl_data["RT(ms)"] = ctrl_data["RT(ms)"].astype(float)

g2 = sns.catplot(x="conflict",y="RT(ms)",col='volatility',data=ctrl_data,kind='bar',order=["No","Low","High"],col_order=["Low","High"])
for ax in g2.axes[0]:
    vol = ax.get_title().split(' = ')[1]
    print(vol)
    vol_data = ctrl_data.loc[ctrl_data["volatility"]==vol]
    print(vol_data)
    add_stat_annotation(ax, data=vol_data, x="conflict", y="RT(ms)", order=["No","Low","High"],
                    box_pairs=[("No", "Low"), ("Low", "High"), ("No", "High")],
                    test='Mann-Whitney', text_format='star', loc='outside', verbose=2, line_offset=0.1)
    #ax.set_yscale("log")
    
g2.savefig(figure_dir+"Correct_vs_incorrect_action_RTs_comparison_control.png")

In [ ]:
stim_data = all_conflicts.loc[all_conflicts["condition"].str.contains("Stim")]
stim_data = stim_data.dropna()
stim_data["RT(ms)"] = stim_data["RT(ms)"].astype(float)
d1_stim_data = stim_data.loc[stim_data["condition"]=="D1-Stim"]
d2_stim_data = stim_data.loc[stim_data["condition"]=="D2-Stim"]

print(np.unique(stim_data["condition"]))
dat_type = ["d1_stimulation","d2_stimulation"]

for i,sd in enumerate([d1_stim_data,d2_stim_data]):

    g2 = sns.catplot(x="conflict",y="RT(ms)",col='volatility',data=sd,kind='bar',order=["No","Low","High"],col_order=["Low","High"])
    for ax in g2.axes[0]:
        vol = ax.get_title().split(' = ')[1]
        print(vol)
        vol_data = sd.loc[sd["volatility"]==vol]

        add_stat_annotation(ax, data=vol_data, x="conflict", y="RT(ms)", order=["No","Low","High"],
                        box_pairs=[("No", "Low"), ("Low", "High"), ("No", "High")],
                        test='Mann-Whitney', text_format='star', loc='outside', verbose=2, line_offset=0.1)
        #ax.set_yscale("log")

    g2.savefig(figure_dir+"Correct_vs_incorrect_action_RTs_comparison_"+dat_type[i]+".png")

In [ ]:
all_conflicts
